#### Importamos librerias necesarias

In [4]:
# Importamos librerias necesiras
import numpy as np
import pandas as pd
import os
import requests
import locale
import calendar
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import mode
from scipy.spatial.distance import cdist # Para calcular distancias entre coordenadas 
from datetime import datetime
from geopy.geocoders import Nominatim
import utm
import openmeteo_requests

import joblib
import category_encoders as ce


locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

pd.set_option('display.max_columns', None)  # Mostrar todas las columnas de los df

#### Asignamos la ruta de directorios

In [2]:
#Asignamos el directorio activo
dirActivo = os.getcwd()
# Creamos una carpeta para que contenga a nuestro dataset
if not os.path.isdir('datos'):os.mkdir('datos')
path_datos = os.path.join(dirActivo+'//datos//')
if not os.path.isdir('datos_procesados'):os.mkdir('datos_procesados')
path_datos_procesados = os.path.join(dirActivo+'//datos_procesados//')

### Cargamos los csv de accidentes ubicado en la carpeta de datos

###### Definimos la funcion de carga de datos

In [ ]:
def CargarDatos(añoIni, añoFin,nombre,extension = 'csv'):
    """
    Carga los datos de accidentalidad   
    y los guarda en un archivo csv
    añoIni: año inicial de los datos
    añoFin: año final de los datos
    nombre: nombre del archivo csv
    """

    # Lista para almacenar los DataFrames de cada archivo
    dfs = []

    # Iterar sobre los años
    while añoIni <= añoFin:
        # Construir el nombre del archivo para el año actual
        nombre_archivo = f'{añoIni}_{nombre}.{extension}'
        
        # Intentar cargar el archivo en un DataFrame
        try:
            df_temporal = pd.read_csv(path_datos+nombre_archivo,sep=";",skipinitialspace=True,decimal=',')
            # Agregar el DataFrame a la lista
            dfs.append(df_temporal)
        except FileNotFoundError:
            print(f"El archivo {nombre_archivo} no existe.")
        
        # Incrementar el año actual
        añoIni += 1

    # Concatenar los DataFrames en uno solo
    df = pd.concat(dfs, ignore_index=True)
    #Quitamos las columnas que se crean vacias
    df = df.iloc[:, :-2]

    # Retornar el DataFrame concatenado de todos los años
    return df



###### Cargamos los datos de Accidentes

In [ ]:
#Cargamos los datos
df = CargarDatos(2019, 2024, 'Accidentalidad')
df.head()

### Cargamos el calendario laboral

In [ ]:
#Cargamos la información del fichero calendario
calendario = pd.read_csv(path_datos+"calendario.csv", sep=";",skipinitialspace=True)
#Quitamos las columnas que se crean vacias
calendario = calendario.iloc[:, :-2]

#tratamos el calendario
reemplazos_TipoFestivo = {
    "de la ": "",
    "traslado ": "",
    'local ciudad de Madrid': "Local",
    re.compile(r'^fiesta', re.IGNORECASE): 'Festivo',
    re.compile(r'^festivo comunidad de madrid', re.IGNORECASE): 'Festivo Comunidad',
    re.compile(r'^Festivo nacional$', re.IGNORECASE): 'Festivo Nacional',
    re.compile(r'^Festivo local$', re.IGNORECASE): 'Festivo Local',
}

reemplazos_Tipo = {
    "festivo": "Festivo",
    "laborable": "Laborable"
}

for patron, nuevo_valor in reemplazos_TipoFestivo.items():
    calendario['Tipo de Festivo'] = calendario['Tipo de Festivo'].str.replace(patron, nuevo_valor, regex=True)

calendario['laborable / festivo / domingo festivo'] = calendario['laborable / festivo / domingo festivo'].replace(reemplazos_Tipo)
    
festivos = calendario[calendario['laborable / festivo / domingo festivo'].str.contains('festivo|Festivo', case=False, na=False)]
festivos
print(festivos['Tipo de Festivo'].unique())
print(festivos['laborable / festivo / domingo festivo'].unique())

#### Unimos el calendario laboral al listado de accidentes

In [ ]:
df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
festivos['Dia'] = pd.to_datetime(festivos['Dia'], format='%d/%m/%Y')
# Combinar los DataFrames en función de las columnas de fecha 'dia' y 'fecha'
df = pd.merge(df, festivos, left_on='fecha', right_on='Dia', how='left')

# Llenar los valores NaN en la columna 'festivo' con 'laborable'
df['laborable / festivo / domingo festivo'].fillna('Laborable', inplace=True)

# Asignar 'fin de semana' si el día es sábado o domingo
df.loc[df['fecha'].dt.dayofweek.isin([5, 6]), 'laborable / festivo / domingo festivo'] = 'Fin de semana'


# Ahora df contiene todos los días con los valores filtrados asignados
df
df.drop(columns=['Dia','Dia_semana'], inplace=True)

#### Ponemos el día de la semana

In [ ]:
# Aplicar la función a la columna 'fecha' para obtener el nombre del día de la semana
df['dia_semana'] = df['fecha'].dt.strftime('%A')

# Imprimir el DataFrame resultante
df['dia_semana'].unique()

df['dia_semana'] = df['dia_semana'].apply(lambda x: x.encode('latin1').decode('utf-8'))
df['dia_semana'].unique()

### Revisamos los valores nulos por columnas

In [ ]:
# Calcular la cantidad de valores nulos en cada columna
valores_nulos_por_columna = df.isnull().sum()

# Calcular el porcentaje de valores nulos en cada columna
porcentaje_nulos_por_columna = (valores_nulos_por_columna / len(df)) * 100

# Crear un DataFrame para mostrar los resultados
resultados_nulos = pd.DataFrame({'Valores Nulos': valores_nulos_por_columna, 'Porcentaje': porcentaje_nulos_por_columna})

# Filtrar solo las columnas que tienen valores nulos
resultados_nulos = resultados_nulos[resultados_nulos['Valores Nulos'] > 0]

print("Columnas con valores nulos:")
print(resultados_nulos.sort_values('Porcentaje', ascending=False))

#Generamos graficos valores nulos
fig = px.imshow(df.isnull(), 
                labels=dict(color="Valores Nulos"),
                title="Valores Nulos")
fig.show()

#### Estado meteorológico

Haciendo uso de la API de Open-Meteo tenemos opción de consultar datos históricos y datos futuros
no detecta las distintas coordenadas por lo tanto unificamos datos de consulta a Madrid para imputar datos vacíos

* Latitud : 40.4165
* Longitud : -3.70256

In [ ]:
# Unificar los valores
mapeo_valores = {
    'Lluvia débil': 'Lluvia',
    'LLuvia intensa': 'Lluvia',
    'Lluvia intensa': 'Lluvia'
    # Aquí puedes agregar más mapeos si es necesario
}
df['estado_meteorológico'] = df['estado_meteorológico'].replace(mapeo_valores)

estado_meteorológico_anterior = df['estado_meteorológico'].unique()
estado_meteorológico_anterior


In [ ]:
weather_description = {
    0: 'Despejado',
    1: "Despejado",
    2: "Nublado",
    3: "Nublado",
    45: "Niebla",
    48: "Niebla con depósito de escarcha",
    51: "Lluvia", #Lluvia débil
    53: "Lluvia", #Lluvia débil
    55: "Lluvia", #Lluvia débil
    56: "Llovizna helada ligera",
    57: "Llovizna helada de intensidad densa",
    61: "Lluvia",  #Lluvia débil
    63: "Lluvia",  #Lluvia moderada
    65: "Lluvia",  #Lluvia intensa
    66: "Lluvia helada ligera",
    67: "Lluvia helada de intensidad fuerte",
    71: "Nevando",
    73: "Nevando",
    75: "Nevando",
    77: "Nevando",
    80: "Lluvia",   #Lluvia intensa
    81: "Lluvia",   #Lluvia intensa
    82: "Lluvia",   #Lluvia intensa
    85: "Nevando",
    86: "Nevando",
    95: "Tormenta",
    96: "Granizando",
    99: "Granizando"
}


In [ ]:
# Asignamos las Fecha inicial y Fecha final para la consulta
fecha_ini = df['fecha'].min().strftime('%Y-%m-%d')
fecha_fin = df['fecha'].max().strftime('%Y-%m-%d')
print(f"Fecha Inicial: {fecha_ini} fecha Final: {fecha_fin}")

# Instanciamos el clientes
openmeteo = openmeteo_requests.Client()

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.4165,
	"longitude": -3.70256,
	"start_date": fecha_ini,
	"end_date": fecha_fin,
	"hourly": "weather_code",
	"daily": "weather_code",
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
# print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
# print(f"Elevation {response.Elevation()} m asl")
# print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
# print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_weather_code = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["weather_code"] = hourly_weather_code

hourly_dataframe = pd.DataFrame(data = hourly_data)
# print(f'{hourly_dataframe=}')

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code

daily_dataframe = pd.DataFrame(data = daily_data)
# print(f'{daily_dataframe=}')

# Agregar descripciones a los DataFrames
hourly_dataframe['weather_description'] = hourly_dataframe['weather_code'].map(weather_description)
daily_dataframe['weather_description'] = daily_dataframe['weather_code'].map(weather_description)

# Convertir la columna 'fecha_hora' a tipo datetime
hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'])

# Crear columnas separadas para fecha y hora
hourly_dataframe['fecha'] = hourly_dataframe['date'].dt.date
hourly_dataframe['hora'] = hourly_dataframe['date'].dt.time

# Eliminar la columna 'fecha_hora' si ya no la necesitas
hourly_dataframe.drop(columns=['date'], inplace=True)
hourly_dataframe


In [ ]:
hourly_dataframe['weather_description'].unique()

In [ ]:
#Formateomops la columna Hora
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce')
df['hora'] = df['hora'].dt.time

# Construir un diccionario que mapee la combinación de fecha y hora a weather_description
weather_dict = {}
for index, row in hourly_dataframe.iterrows():
    fecha_hora_redondeada = row['fecha'].strftime('%Y-%m-%d'), row['hora'].hour
    weather_dict[fecha_hora_redondeada] = row['weather_description']

# Actualizar el DataFrame original utilizando el diccionario
condicion = (df['estado_meteorológico'].isnull()) | (df['estado_meteorológico'] == 'Se desconoce')
indices_a_actualizar = df.index[condicion]
df.loc[indices_a_actualizar, 'estado_meteorológico'] = df.loc[indices_a_actualizar].apply(lambda row: weather_dict.get((row['fecha'].strftime('%Y-%m-%d'), row['hora'].hour), None), axis=1)


#Mostramos los valores de Antes y Despues 
estado_meteorológico_nuevo = df['estado_meteorológico'].unique()

print(f'{estado_meteorológico_anterior}\n{estado_meteorológico_nuevo}')


#### Lesividad

In [ ]:
anterior =df[['lesividad','cod_lesividad']].value_counts()

# Quitamos los espacios en blanco para despues poder asignar un valor a los NaN
df['lesividad'] = df['lesividad'].str.strip()

# Completamos los valores NaN con 'Sin asistencia sanitaria' y '14' respectivamente
df.loc[(df['lesividad'].isna()) & (df['cod_lesividad'].isna()), 
        ['lesividad', 'cod_lesividad']] = ['Sin asistencia sanitaria', 14]


# Cambiamos los valores de 'Se Desconoce' y '77' a 'Sin asistencia sanitaria' y '14' respectivamente
df.loc[(df['lesividad'] == 'Se desconoce') & (df['cod_lesividad'] == 77),
        ['lesividad', 'cod_lesividad']] = ['Sin asistencia sanitaria', 14]

# Crear un diccionario de clasificación de lesividad
clasificacion_lesividad = {
    1.0: 'Leve',
    2.0: 'Grave',
    3.0: 'Grave',
    4.0: 'Fallecido',
    5.0: 'Leve',
    6.0: 'Leve',
    7.0: 'Leve',
    14.0: 'Muy leve'
    
}

# Aplicar la clasificación de lesividad y crear una nueva columna
df['tipo_lesividad'] = df['cod_lesividad'].apply(lambda x: clasificacion_lesividad.get(x, 'No definido'))

posterior = df[['lesividad','cod_lesividad']].value_counts()

print(f'{anterior=}\n\n{posterior=}')

# Mostrar el DataFrame con la nueva columna
df


In [ ]:
#Agrupamos los datos para ver las clasificaciones con la cantidad y el %
lesividad = df.groupby(['lesividad','cod_lesividad']).size().reset_index(name='cantidad')
suma_total = lesividad['cantidad'].sum()
lesividad['porcentaje'] = (lesividad['cantidad'] / suma_total) * 100
lesividad.sort_values('cod_lesividad', ascending=False, inplace=True)
lesividad

In [ ]:
#Agrupamos los datos para ver las clasificaciones con la cantidad y el %
lesividad = df.groupby(['tipo_lesividad']).size().reset_index(name='cantidad')
suma_total = lesividad['cantidad'].sum()
lesividad['porcentaje'] = (lesividad['cantidad'] / suma_total) * 100
lesividad.sort_values('cantidad', ascending=False, inplace=True)
lesividad

#### Alcohol y Drogas

In [ ]:
# Rellenar NaN en 'positiva_alcohol' con 'N' y convertir a valores numéricos
df['positiva_alcohol'] = df['positiva_alcohol'].fillna('N').replace({'N': 0, 'S': 1})
 
# Reemplazar NaN por 0 en la columna 'positiva_droga'
df['positiva_droga'].fillna(0, inplace=True)
 
# Filtrar filas donde se detectó alcohol y drogas simultáneamente
ambos_detectados = df[(df['positiva_alcohol'] == 1) & (df['positiva_droga'] == 1)]
 
# Contar el número de casos donde se detectó alcohol y drogas simultáneamente
num_ambos_detectados = len(ambos_detectados)
 
print(f"Número de casos donde se detectó tanto alcohol como drogas: {num_ambos_detectados}")
 
# Calcular el total de casos donde se detectó alcohol
total_alcohol = df['positiva_alcohol'].sum()
 
# Calcular el total de casos donde se detectaron drogas
total_drogas = df['positiva_droga'].sum()
 
# Calcular el total de casos donde se detectó tanto alcohol como drogas
total_ambos = len(ambos_detectados)
 
# Calcular el porcentaje de casos donde se detectó tanto alcohol como drogas con respecto al total de casos
porcentaje_ambos = (total_ambos / len(df)) * 100
 
print(f"Porcentaje de casos donde se detectó tanto alcohol como drogas: {porcentaje_ambos:.2f}%")

#### Tipo Persona

In [ ]:
df[df['tipo_persona'].isnull()]

In [ ]:
# Revisamos los expedientes en los que aparece el tipo persona vacio

# Crear una lista de valores únicos de 'num_expediente' para las filas donde 'tipo_persona' es nulo
expedientes = df[df['tipo_persona'].isnull()]['num_expediente'].unique()

# Crear una máscara booleana que indica si cada valor en la columna 'num_expediente' está en la lista de valores a filtrar
mascara_filtro = df['num_expediente'].isin(expedientes)

# Aplicar la máscara booleana al DataFrame para filtrar las filas
df_filtrado = df[mascara_filtro]
df_filtrado

#### Pasajero

In [ ]:
# Como en todos los expientes tenemos conductores vamos a comletar los datos vacios con pasajero
df.loc[df['tipo_persona'].isnull(), 'tipo_persona'] = 'Pasajero'

# Crear una máscara booleana que indica si cada valor en la columna 'num_expediente' está en la lista de valores a filtrar
mascara_filtro = df['num_expediente'].isin(expedientes)

# Aplicar la máscara booleana al DataFrame para filtrar las filas
df_filtrado = df[mascara_filtro]
df_filtrado



#### Tipo Accidente
 


In [ ]:
conductores = df[(df['tipo_persona'] == 'Conductor') & (df['tipo_accidente'].notna())]

# Calcular el número de vehículos implicados por expediente
implicados = conductores.groupby(['distrito', 'num_expediente', 'tipo_accidente'])['tipo_vehiculo'].count().reset_index()
implicados = implicados.rename(columns={'tipo_vehiculo': 'implicados'})

# Calcular la moda del tipo de accidente por distrito y número de vehículos implicados
moda_por_grupo = implicados.groupby(['distrito', 'implicados'])['tipo_accidente'].agg(lambda x: x.mode()[0]).reset_index()

moda_por_grupo

# Filtrar los registros vacíos en la columna 'tipo_accidente'
vacios = df[df['tipo_accidente'].isnull()]

# Agrupar por número de expediente y calcular el número de vehículos implicados para cada expediente
vacios = vacios.groupby(['distrito', 'num_expediente']).size().reset_index(name='implicados')


for index, row in vacios.iterrows():

    #Buscamos el tipo de accidente por Distrito y Número implicados

    tipo_accidente = moda_por_grupo[(moda_por_grupo['distrito'] == row['distrito']) & (moda_por_grupo['implicados'] == row['implicados'])]['tipo_accidente']

        # Verificar si se encontró un tipo de accidente en moda_por_grupo
    if not tipo_accidente.empty:        
        # Asignar el tipo de accidente en df para el número de expediente correspondiente
        df.loc[(df['num_expediente'] == row['num_expediente']) & (df['tipo_accidente'].isnull()), 'tipo_accidente'] = tipo_accidente.iloc[0]

# Unificamos datos 
#Unificamos los valores de la columna tipo_accidente
reemplazos_TipoAccidente = {
    'Colisión múltiple':'Colisión',
    'Colisión fronto-lateral':'Colisión',
    'Colisión lateral':'Colisión',
    'Colisión frontal':'Colisión',
    'Alcance':'Colisión',
    'Atropello a persona':'Atropello',
    'Atropello a animal':'Atropello',
    'Despeñamiento':'Caída',
    'Choque contra obstáculo fijo': 'Colisión',
    'Solo salida de la vía': 'Otro',
    'Otro':'Otro'
}

df['tipo_accidente'] = df['tipo_accidente'].replace(reemplazos_TipoAccidente)
df['tipo_accidente'].value_counts()


#### Revisamos valores nulos

In [ ]:
# Calcular la cantidad de valores nulos en cada columna
valores_nulos_por_columna = df.isnull().sum()

# Calcular el porcentaje de valores nulos en cada columna
porcentaje_nulos_por_columna = (valores_nulos_por_columna / len(df)) * 100

# Crear un DataFrame para mostrar los resultados
resultados_nulos = pd.DataFrame({'Valores Nulos': valores_nulos_por_columna, 'Porcentaje': porcentaje_nulos_por_columna})

# Filtrar solo las columnas que tienen valores nulos
resultados_nulos = resultados_nulos[resultados_nulos['Valores Nulos'] > 0]

print("Columnas con valores nulos:")
print(resultados_nulos.sort_values('Porcentaje', ascending=False))

#Generamos graficos valores nulos
fig = px.imshow(df.isnull(), 
                labels=dict(color="Valores Nulos"),
                title="Valores Nulos")
fig.show()

### Procesamiento de columnas

#### Creamos Latitud y Longitud desde UTM

##### Hacemos la conversión

In [ ]:

# Función para convertir coordenadas UTM a longitud y latitud
def utm_to_latlon(x_utm, y_utm):
    if isinstance(x_utm, str) and isinstance(y_utm, str):
        try:
            # Reemplazar la coma por un punto y luego convertir a flotante
            x_utm = float(x_utm.replace(',', '.'))
            y_utm = float(y_utm.replace(',', '.'))

            if 100000 <= x_utm <= 999999 and 0 <= y_utm <= 10000000:
                lat, lon = utm.to_latlon(x_utm, y_utm, zone_number=30, northern=True)
                return lat, lon
            else :
                return None, None

        except ValueError:
            # Si hay un error al convertir, devolver None para latitud y longitud
            return None, None
    else:
        return None, None
    

# Aplicar la función a las columnas 'coordenada_x_utm' y 'coordenada_y_utm' para crear nuevas columnas 'latitud' y 'longitud'
df['latitud'], df['longitud'] = zip(*df.apply(lambda row: utm_to_latlon(row['coordenada_x_utm'], row['coordenada_y_utm']), axis=1))

# Mostrar las primeras filas del DataFrame con las nuevas columnas de coordenadas geográficas
df.head()



##### Completamos con la moda por distrito si alguna de ellas no se ha podido completar

In [ ]:
# Eliminar filas con valores no válidos en longitud y latitud
df_valid = df.dropna(subset=['longitud', 'latitud'])
 
# Calcular la moda de longitud y latitud para cada distrito que tenga datos disponibles
mode_longitud = df_valid.groupby('distrito')['longitud'].apply(lambda x: mode(x)[0] if len(x) > 0 else np.nan).to_dict()
mode_latitud = df_valid.groupby('distrito')['latitud'].apply(lambda x: mode(x)[0] if len(x) > 0 else np.nan).to_dict()
 
# Llenar los valores faltantes con la moda
df['longitud'] = df['longitud'].fillna(df['distrito'].map(mode_longitud))
df['latitud'] = df['latitud'].fillna(df['distrito'].map(mode_latitud))
 
# Mostrar las primeras filas del DataFrame con las nuevas coordenadas geográficas llenadas
df.head()

#### Tipo Vehiculo

In [ ]:
#Guardamos los valores anteriores
anterior = df.tipo_vehiculo.unique()

# Rellenamos los NaN como sin especificar
df['tipo_vehiculo'].fillna('Sin especificar', inplace=True)

# Generamos el diccionario para unificar valores
reemplazos_TipoVehiculo = {
    'Motocicleta > 125cc':'Motocicleta',
    'Ciclomotor':'Motocicleta',
    'Motocicleta hasta 125cc':'Motocicleta',
    'Todo terreno':'Turismo',
    'Camión rígido':'Vehiculo Pesado',
    'Maquinaria de obras':'Vehiculo Pesado',
    'Tractocamión' :'Vehiculo Pesado',
    'Cuadriciclo no ligero':'Motocicleta',
    'Vehículo articulado':'Vehiculo Pesado',
    'Autobús articulado' :'Autobús',
    'Otros vehículos con motor':'Otros',
    'Patinete' :'Otros',
    'Ciclo':'Motocicleta',
    'Cuadriciclo ligero':'Motocicleta',
    'VMU eléctrico':'Turismo',
    'Semiremolque':'Semiremolque',
    'Microbús <= 17 plazas':'Autobús',
    'Autobus EMT':'Autobús',
    'Otros vehículos sin motor':'Otros',
    'Bicicleta EPAC (pedaleo asistido)':'Bicicleta',
    'Bicicleta EPAC (pedaleo asistido)':'Bicicleta',
    'Moto de tres ruedas > 125cc':'Motocicleta',
    'Ambulancia SAMUR':'Ambulancia',
    'Moto de tres ruedas hasta 125cc':'Motocicleta',
    'Ciclomotor de dos ruedas L1e-B':'Motocicleta',
    'Maquinaria agrícola' :'Vehiculo Pesado',
    'Autobús articulado EMT':'Autobús',
    'Autobús EMT':'Autobús',
    'Motocicleta de motor L1e-A':'Motocicleta',
    'Ciclomotor de tres ruedas':'Motocicleta',
    'Ciclo de motor L1e-A':'Motocicleta',
    'Patinete no eléctrico':'Otros',
    'Motocicleta de dos ruedas L1e-B':'Motocicleta',
    'Motocicleta de tres ruedas':'Motocicleta',
    'Otros no eléctrico':'Otros',
    'Bicicleta EPAC (pedaleo asistido)':'Bicicleta',
    'Furgoneta':'Turismo',
    'Caravana':'Turismo',
    'Autocaravana':'Turismo',
    'Semiremolque':'Turismo',
    'Remolque':'Turismo',
    'Camión de bomberos':'Emergencias',
    'Ambulancia':'Emergencias',
    'Tranvía':"Tren/metro",
    'Sin especificar':'Otros'
}

# Unificamos los vehiculos segun la definicion en el diccionario
df['tipo_vehiculo'] = df['tipo_vehiculo'].replace(reemplazos_TipoVehiculo)

# Guardamos los datos despues de la reclasificación
posterior = df.tipo_vehiculo.unique()

#Mostramos los valores antes y despues de la unificación
print(f'{anterior=} \n\n {posterior=}')

del anterior , posterior


#### Localización

In [ ]:
# # Función para eliminar el número de calle después de la coma
# def eliminar_numero_despues_de_coma(localizacion):
#     if ',' in localizacion:
#         return localizacion.split(',')[0].strip()  # Obtener la parte antes de la coma y eliminar espacios en blanco adicionales
#     else:
#         return localizacion  # Si no hay coma, devolver la localización sin cambios
 
# # Aplicar la función a la columna 'localizacion' para eliminar el número de calle después de la coma
# df['localizacion'] = df['localizacion'].apply(eliminar_numero_despues_de_coma)
 
# # Mostrar las primeras filas del DataFrame con la columna 'localizacion' modificada
# df.head()

#### Tipo de via

In [ ]:
# # Función para identificar el tipo de vía
# def identificar_tipo_via(localizacion):
#     # Expresión regular para buscar los patrones específicos
#     patron_cruce = r'\/'
#     patron_m30 = r'M-30|M -30'
#     patron_a1= r'A-1'
#     patron_a2= r'A-2'
#     patron_a3= r'A-3'
#     patron_a4= r'A-4'
#     patron_a5= r'A-5'
#     patron_a6= r'A-6'
#     patron_m40= r'M-40|M -40'
#     patron_plaza = r'plaza|PLAZA|PLZ.|Atocha'
#     patron_calle = r'call|cañada|cñada|custa|km|bulev|ronda|alcala|c/|calle|CALL.|GRAN VIA|SANTA ENGRACIA|C '
#     patron_paseo = r'paseo'
#     patron_avenida = r'avenida|avda|avd|av'
#     patron_carretera = r'crta|carretera|CTRA.|CARRETERA|CRA.' #no se por que me estaá dando error CARRETERA y CTRA:
#     patron_glorieta = r'GTA.|GLORIETA|glorieta|glta'
#     patron_camino = r'cmno|camino|CAMINO|TRVA.'
#     patron_tunel = r'túnel|TUNEL|TUNEL.|tunel |subterraneo'
#     patron_autovia = r'M-301|AUTOV.'
#     patro_aeropuerto = r'Aeropuerto|Aerop'
#     patron_autopista = r'A-42'
 
#     # Buscar el patrón de cruce, autovía, calle, paseo, avenida.... y si existe el patron devolver lo que se le pida
#     match_cruce = re.search(patron_cruce, localizacion)
#     if match_cruce:
#         return 'Cruce'
#     match_m30 = re.search(patron_m30, localizacion, flags=re.IGNORECASE)
#     if match_m30:
#         return 'M-30'  
#     match_a1 = re.search(patron_a1, localizacion, flags=re.IGNORECASE)
#     if match_a1:
#         return 'A-1'
#     match_a2 = re.search(patron_a2, localizacion, flags=re.IGNORECASE)
#     if match_a2:
#         return 'A-2'
#     match_a3 = re.search(patron_a3, localizacion, flags=re.IGNORECASE)
#     if match_a3:
#         return 'A-3'
#     match_a4 = re.search(patron_a4, localizacion, flags=re.IGNORECASE)
#     if match_a4:
#         return 'A-4'
#     match_a5 = re.search(patron_a5, localizacion, flags=re.IGNORECASE)
#     if match_a5:
#         return 'A-5'
#     match_a6 = re.search(patron_a6, localizacion, flags=re.IGNORECASE)
#     if match_a6:
#         return 'A-6'
#     match_m40 = re.search(patron_m40, localizacion, flags=re.IGNORECASE)
#     if match_m40:
#         return 'M-40'   
#     match_plaza = re.search(patron_plaza, localizacion, flags=re.IGNORECASE)
#     if match_plaza:
#         return 'Plaza'  
#     match_calle = re.search(patron_calle, localizacion, flags=re.IGNORECASE)
#     if match_calle:
#         return 'Calle'
#     match_paseo = re.search(patron_paseo, localizacion, flags=re.IGNORECASE)
#     if match_paseo:
#         return 'Paseo'
#     match_avenida = re.search(patron_avenida, localizacion, flags=re.IGNORECASE)
#     if match_avenida:
#         return 'Avenida'
#     match_carretera = re.search(patron_carretera, localizacion, flags=re.IGNORECASE)
#     if match_carretera:
#         return 'Carretera'
#     match_glorieta = re.search(patron_glorieta, localizacion, flags=re.IGNORECASE)
#     if match_glorieta:
#         return 'Glorieta'  
#     match_camino = re.search(patron_camino, localizacion, flags=re.IGNORECASE)
#     if match_camino:
#         return 'Camino'
#     match_tunel = re.search(patron_tunel, localizacion, flags=re.IGNORECASE)
#     if match_tunel:
#         return 'Túnel'
#     match_autovia = re.search(patron_autovia, localizacion, flags=re.IGNORECASE)
#     if match_autovia:
#         return 'Autovía'
#     match_aeropuerto = re.search(patro_aeropuerto, localizacion, flags=re.IGNORECASE)
#     if match_aeropuerto:
#         return 'Aeropuerto'
#     match_autopista = re.search(patron_autopista, localizacion, flags=re.IGNORECASE)
#     if match_autopista:
#         return 'Autopista'
    
#     return 'Calle'  # Si no se encuentra ningún patrón, devolver 'desconocido'

# # Aplicar la función a la columna 'localizacion' para crear la nueva columna 'tipo_via'
# df['tipo_via'] = df['localizacion'].apply(identificar_tipo_via)

# # Mostrar las primeras filas del DataFrame con la nueva columna 'tipo_via'
# df.head()

# df['tipo_via'].value_counts()

#### Ponemos el nombre de la calle y el distrito por los puntos mas cercanos en el callejero

In [ ]:
callejero = pd.read_csv(f'{path_datos}callejeroMadrid.csv',encoding='iso-8859-1',delimiter=';',decimal=',')
distritos = pd.read_excel(f'{path_datos}distritos.xlsx')

# accidentes = pd.read_csv(f'{path_datos_procesados}accidentes_procesados.csv')




# Función para convertir coordenadas geográficas en formato de grados, minutos y segundos a decimal
# Función para convertir coordenadas geográficas en formato de grados, minutos y segundos a decimal
def dms_to_decimal(coordenada):
    # Expresión regular para extraer los componentes de la coordenada
    match = re.match(r"([-+]?\d+)°(\d+)'([\d.]+)''\s*([NSWE])", coordenada)
    if match:
        grados, minutos, segundos, direccion = match.groups()
        decimal = float(grados) + float(minutos)/60 + float(segundos)/3600
        if direccion in ['S', 'W']:
            decimal = -decimal
        return decimal
    else:
        return None


callejero['LATITUD'] = callejero['LATITUD'].apply(lambda x: dms_to_decimal(x) if isinstance(x, str) else x)
callejero['LONGITUD'] = callejero['LONGITUD'].apply(lambda x: dms_to_decimal(x) if isinstance(x, str) else x)
callejero.rename(columns={'LATITUD': 'latitud', 'LONGITUD': 'longitud'}, inplace=True)
callejero

In [ ]:
# Función para calcular la distancia euclidiana entre dos puntos
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Inicializar el diccionario para almacenar información por expediente
info_por_expediente = {}

# Agrupar los accidentes por número de expediente
for num_expediente, grupo in df.groupby('num_expediente'):
    latitudes_accidentes = grupo['latitud'].values.reshape(-1, 1)
    longitudes_accidentes = grupo['longitud'].values.reshape(-1, 1)
    coordenadas_accidentes = np.hstack((latitudes_accidentes, longitudes_accidentes))
    
    # Calcular las distancias a todas las coordenadas del callejero
    distancias = cdist(coordenadas_accidentes, callejero[['latitud', 'longitud']], metric='euclidean')
    
    # Encontrar el índice de la calle más cercana para cada accidente del expediente
    indices_cercanos = distancias.argmin(axis=1)
    
    # Obtener el nombre y la clase de la calle más cercana para cada accidente
    calles_cercanas = callejero.loc[indices_cercanos, ['VIA_NOMBRE_ACENTOS', 'VIA_CLASE', 'DISTRITO']]
    
    # Tomar la primera calle más cercana para representar el expediente
    calle_mas_cercana = calles_cercanas.iloc[0]
    
    # Guardar la información en el diccionario por número de expediente
    info_por_expediente[num_expediente] = calle_mas_cercana

# Mapear las calles más cercanas a cada expediente en el DataFrame de accidentes
df['callejero_calle'] = df['num_expediente'].map(lambda x: info_por_expediente[x]['VIA_NOMBRE_ACENTOS'])
df['callejero_tipo_via'] = df['num_expediente'].map(lambda x: info_por_expediente[x]['VIA_CLASE'])
df['callejero_distrito'] = df['num_expediente'].map(lambda x: info_por_expediente[x]['DISTRITO'])


# Renombrar las columnas para que coincidan
distritos.rename(columns={'COD_DIS': 'callejero_distrito', 'DISTRI_MT': 'distrito_mt'}, inplace=True)

# Fusionar los DataFrames por la columna 'cod_distrito' y agregar solo la columna 'distrito_mt'
df = pd.merge(df, distritos[['callejero_distrito', 'distrito_mt']], on='callejero_distrito', how='left')
df

#### Asignamos tramo horario

In [ ]:
#Convertimos la columna hora en hora
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce')
df['hora'] = df['hora'].dt.time

# Definimos la función para asignar la franja horaria
def asignar_franja_horaria(hora):
    if hora.hour >= 0 and hora.hour < 7:
        tramo = 'Madrugada'
    elif hora.hour >= 7 and hora.hour < 14:
        tramo = 'Mañana'
    elif hora.hour >= 14 and hora.hour < 21:
        tramo ='Tarde'
    else:
        tramo ='Noche'
    return tramo 
# Creamos una columna nueva para la franja horaria
df['franja_horaria'] = df['hora'].apply(asignar_franja_horaria).apply(pd.Series)

df

#### Asignamos tramos de edad

In [ ]:
# Creamos un DataFrame auxiliar para facilitar el cálculo de la moda
edades_df = df.copy()

# Reemplazamos 'Desconocido' con NaN para que no se considere en el cálculo de la moda
edades_df['rango_edad'].replace('Desconocido', np.nan, inplace=True)

# Agrupamos por tipo de vehículo y tipo de persona y calculamos la moda de rango_edad
moda_por_grupo = edades_df.groupby(['tipo_vehiculo', 'tipo_persona'])['rango_edad'].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan).reset_index()

# Iteramos sobre el DataFrame original para asignar las edades desconocidas
for index, row in df.iterrows():
    if row['rango_edad'] == 'Desconocido':
        tipo_vehiculo = row['tipo_vehiculo']
        tipo_persona = row['tipo_persona']
        moda = moda_por_grupo[(moda_por_grupo['tipo_vehiculo'] == tipo_vehiculo) & (moda_por_grupo['tipo_persona'] == tipo_persona)]['rango_edad'].values
        if moda:
            df.at[index, 'rango_edad'] = moda[0]

# Verificamos el conteo de valores después de la corrección
df['rango_edad'].value_counts()

# Mapeamos los rangos de edad a los grupos definidos
edad_reemplazo = {
    'De 45 a 49 años': '40-49',
    'De 30 a 34 años': '30-39',
    'De 40 a 44 años': '40-49',
    'De 65 a 69 años': '65-69',
    'Más de 74 años': '+74',
    'De 21 a 24 años': '18-29',
    'De 35 a 39 años': '30-39',
    'De 50 a 54 años': '50-59',
    'De 60 a 64 años': '60-64',
    'De 55 a 59 años': '50-59',
    'De 15 a 17 años': '0-17',
    'De 18 a 20 años': '18-29',
    'De 25 a 29 años': '18-29',
    'De 70 a 74 años': '70-74',
    'De 6 a 9 años': '0-17',
    'Menor de 5 años': '0-17',
    'De 10 a 14 años': '0-17'
}

# Creamos una nueva columna que contenga las edades agrupadas
df['grupo_edad'] = df['rango_edad'].replace(edad_reemplazo)



#### Sexo

In [ ]:
# Reemplazar 'Desconocido' con NaN para que no se considere en el cálculo de la moda
df['sexo'].replace('Desconocido', np.nan, inplace=True)

# Define las agrupaciones iniciales
agrupaciones = ['tipo_vehiculo', 'tipo_persona', 'tipo_accidente', 'grupo_edad']

# Define una lista para almacenar las agrupaciones que se probarán
agrupaciones_a_probar = agrupaciones.copy()

# Itera sobre las agrupaciones a probar
while len(agrupaciones_a_probar) > 0:
    # Calcula la moda de sexo por las agrupaciones actuales
    moda_sexo = df.groupby(agrupaciones_a_probar)['sexo'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    # Imputa los valores de moda en las filas donde el sexo es NaN
    df['sexo'] = df.apply(lambda row: moda_sexo.get(tuple(row[agrupaciones_a_probar]), row['sexo']) if pd.isna(row['sexo']) else row['sexo'], axis=1)

    # Verifica si todavía hay valores NaN en el sexo
    if df['sexo'].isna().sum() == 0:
        # Si no hay valores NaN, la imputación se completó, termina el bucle
        print(f"Imputación completada con las agrupaciones: {agrupaciones_a_probar}")
        break
    else:
        # Si todavía hay valores NaN, elimina una columna de las agrupaciones y vuelve a intentarlo
        print(f"Imputación incompleta con las agrupaciones: {agrupaciones_a_probar} valores vacíos {df['sexo'].isna().sum()}")
        agrupaciones_a_probar.pop()

# Verifica si la imputación fue completamente exitosa
if df['sexo'].isna().sum() > 0:
    print(f"Imputación final incompleta. Quedan {df['sexo'].isna().sum()} valores vacíos.")
else:
    print("Imputación final completada.")

In [ ]:
# Agrupar por número de expediente y sumar el total de conductores, pasajeros y peatones
total_por_expediente = df.groupby('num_expediente')['tipo_persona'].value_counts().unstack(fill_value=0)
total_por_expediente = total_por_expediente.rename(columns={1: 'Conductor', 2: 'Pasajero', 3: 'Peatón'})
total_por_expediente['Total'] = total_por_expediente['Conductor'] + total_por_expediente['Peatón']

# Mostrar el resultado
print(total_por_expediente)

In [ ]:
df.info()

### Guardamos los datos procesados

In [ ]:
df.to_csv(path_datos_procesados+'accidentesCalles_procesados.csv',index=False)

In [ ]:
df = pd.read_csv(path_datos_procesados+'accidentesCalles_procesados.csv')

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
try:
    df.drop(columns=['localizacion','numero','cod_distrito','distrito','rango_edad','cod_lesividad','lesividad','coordenada_x_utm',
                    'coordenada_y_utm','Tipo de Festivo', 'Festividad','callejero_distrito'], inplace=True)
    df.columns
except:
    pass

In [ ]:
# Renombrar las columnas del DataFrame df
df = df.rename(columns={
    'num_expediente': 'Expediente',
    'fecha': 'Fecha',
    'dia_semana':'Día semana',
    'laborable / festivo / domingo festivo':'Tipo día',
    'hora': 'Hora',
    'distrito_mt':'Distrito',
    'callejero_tipo_via':'Tipo de vía',
    'callejero_calle':'Calle',
    'latitud':'Latitud',
    'longitud':'Longitud',
    'tipo_accidente': 'Tipo accidente',
    'estado_meteorológico' : 'Estado meteorológico',
    'tipo_vehiculo':'Tipo vehiculo',
    'tipo_persona':'Implicado',
    'grupo_edad':'Edad',
    'sexo':'Sexo',
    'tipo_lesividad' : 'Lesividad',
    'positiva_alcohol':'Positivo alcohol',
    'positiva_droga':'Positivo droga',
    'franja_horaria':'Tramo horario'
    
})

column_order = ['Expediente', 'Fecha','Día semana','Tipo día','Hora','Tramo horario','Distrito','Tipo de vía','Calle','Latitud','Longitud','Tipo accidente',
                'Estado meteorológico','Tipo vehiculo','Implicado','Sexo','Edad','Lesividad','Positivo alcohol','Positivo droga']
df = df.reindex(columns=column_order)

# Ponemos Distrito / Tipo de Vía y Calle como nombre propio
df[['Distrito', 'Tipo de vía', 'Calle']] = df[['Distrito', 'Tipo de vía', 'Calle']].apply(lambda x: x.str.title())

df.head()


In [ ]:
df.to_csv(path_datos_procesados+'accidentes_procesados.csv',index=False)


### Agrupamos los datos procesados por expediente

In [ ]:
# Crear tablas de frecuencia cruzada para cada variable categórica
agrupado = df.groupby(['Expediente','Fecha','Día semana','Tipo día','Hora','Tramo horario','Distrito','Tipo de vía','Calle','Latitud','Longitud','Tipo accidente','Estado meteorológico']).size().reset_index(name='Implicados')
sexo_cruzado = pd.crosstab(index=df['Expediente'], columns=df['Sexo'])
implicado_cruzado = pd.crosstab(index=df['Expediente'], columns=df['Implicado'])
edad_cruzada = pd.crosstab(index=df['Expediente'], columns=df['Edad'])
lesividad_cruzada = pd.crosstab(index=df['Expediente'], columns=df['Lesividad'])
tipo_vehiculo_cruzado = pd.crosstab(index=df['Expediente'], columns=df['Tipo vehiculo'])

accidentes = agrupado.merge(sexo_cruzado, on='Expediente', how='left').merge(implicado_cruzado, on='Expediente', how='left').merge(edad_cruzada, on='Expediente', how='left').merge(lesividad_cruzada, on='Expediente', how='left').merge(tipo_vehiculo_cruzado, on='Expediente', how='left')

accidentes

In [ ]:
accidentes.to_csv(path_datos_procesados+'accidentes_procesados_agrupados.csv',index=False)

### Agrupamos los datos por Distrito y asignamos la latitud y la longitud por distrito

In [ ]:
distritos_madrid = df['Distrito'].unique()
distritos_madrid


# Crear una lista de diccionarios para almacenar los datos
datos_distritos = []

# Inicializar el geocodificador de Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# Obtener las coordenadas para cada distrito
for distrito in distritos_madrid:
    location = geolocator.geocode(f"{distrito}, Madrid, Spain")
    if location:
        datos_distritos.append({
            "Distrito": distrito,
            "Latitud": location.latitude,
            "Longitud": location.longitude
        })
    else:
        print(f"No se pudo encontrar la ubicación para {distrito}")

# Convertir la lista de diccionarios en un DataFrame
df_distritos = pd.DataFrame(datos_distritos)

# Mostrar el DataFrame con las coordenadas de los distritos
print(df_distritos)

accidentesDistrito = df['Distrito'].value_counts().reset_index()
df_final = pd.merge(df_distritos, accidentesDistrito, on='Distrito', how='left')
df_final
df_final.to_csv(f'{path_datos_procesados}accidentes_distritos.csv', index=False)

### Ponemos fecha y hora

In [ ]:
# Asegúrate de que las columnas de fecha y hora estén en formato adecuado
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Hora'] = pd.to_timedelta(df['Hora'])

# Combina la información de fecha y hora en una sola columna de fecha y hora
df['Fecha_Hora'] = df['Fecha'] + df['Hora']

# Muestra el DataFrame con la nueva columna de fecha y hora
print(df[['Fecha', 'Hora', 'Fecha_Hora']])


### Preparamos los datos para ML

!pip install category_encoders

In [ ]:
df = pd.read_csv(path_datos_procesados+'accidentes_procesados.csv')

In [ ]:
dfML = df.copy()

In [ ]:
dfML

#### Codificación ordinal
Asigna a cada categoría un valor numérico único en función de su orden o importancia. Esto es útil cuando las categorías tienen un orden natural, como por ejemplo, "bajo", "medio" y "alto".

##### Codificamos el tramo de edad

In [ ]:
# Renombrar la categoría '+74' a '74+'
dfML['Edad'] = dfML['Edad'].replace('+74', '74+')

# Realizar Ordinal Encoding en la columna 'Edad'
dfML['Edad_codificada'] = dfML['Edad'].astype('category').cat.codes

# Agrupar por 'Edad' y 'Edad_codificada' y contar el número de implicados en cada grupo
agrupado = dfML.groupby(['Edad', 'Edad_codificada']).size().reset_index(name='Implicados')

# Mostrar el DataFrame resultante
print(agrupado)




##### Codificamos Lesividad

In [ ]:
# Define el orden de las categorías de Lesividad en función de su importancia (invertido)
categorias_lesividad = [
    'Muy leve',
    'Leve',
    'Grave',
    'Fallecido'
]

# Crear una instancia de pd.Categorical con el orden especificado
lesividad_categorica = pd.Categorical(dfML['Lesividad'], categories=categorias_lesividad, ordered=True)

# Asignar los códigos resultantes a una nueva columna
dfML['Lesividad_codificada'] = lesividad_categorica.codes

# Agrupar por 'Lesividad' y 'Lesividad_codificada' y contar el número de implicados en cada grupo
agrupado = dfML.groupby(['Lesividad', 'Lesividad_codificada']).size().reset_index(name='Implicados')

# Mostrar el DataFrame resultante
print(agrupado)




#### Codificación sinusoidal

Esta técnica utiliza funciones sinusoidales para codificar variables cíclicas, como días de la semana, horas del día o meses del año. Se basa en la idea de que los datos temporales tienen una naturaleza periódica y pueden ser representados por funciones sinusoidales. La codificación sinusoidal asigna valores que oscilan entre -1 y 1 para capturar la fase de la función sinusoidal en relación con el tiempo.


##### Día semana / Mes Año

In [ ]:
# Asegúrate de que la columna de fecha esté en formato datetime
dfML['Fecha'] = pd.to_datetime(dfML['Fecha'])

# Obtener el día de la semana como un número entero (de 0 a 6 donde 0 es lunes y 6 es domingo)
dia_semana = dfML['Fecha'].dt.dayofweek
mes = dfML['Fecha'].dt.month

# Calcular la codificación sinusoidal de los días de la semana y de los meses
# Utilizamos la función sinusoidal para capturar la naturaleza cíclica de los días de la semana
dfML['Dia_semana_codificado'] = np.sin(2 * np.pi * dia_semana / 7)
# Utilizamos la función sinusoidal para capturar la naturaleza cíclica de los meses del año
dfML['Mes_codificado'] = np.sin(2 * np.pi * mes / 12)
dfML


##### Horas del día

In [ ]:
# Obtener la hora del día como un número entero (de 0 a 23)
dfML['Hora'] = pd.to_datetime(dfML['Hora'])
hora_dia = dfML['Hora'].dt.hour

# Calcular la codificación sinusoidal de las horas del día
# Utilizamos la función sinusoidal para capturar la naturaleza cíclica de las horas del día
dfML['Hora_codificada'] = np.sin(2 * np.pi * hora_dia / 24)
dfML

#### Target encodign
Reemplaza cada categoría con la media (o alguna otra estadística) del valor objetivo correspondiente a esa categoría. Esta técnica puede ser útil cuando hay una relación entre la variable categórica y la variable objetivo.

##### Calle / Tipo de vía / Distrito

In [ ]:
# Inicializar el codificador de frecuencia
frequency_encoder_calle = ce.CountEncoder(cols=['Calle'])
frequency_encoder_via = ce.CountEncoder(cols=['Tipo de vía'])
frequency_encoder_distrito = ce.CountEncoder(cols=['Distrito'])

# Aplicar la codificación de frecuencia a las columnas 'Calle', 'Tipo de vía' y 'Distrito'
dfML['Calle_encoded_frequency'] = frequency_encoder_calle.fit_transform(dfML['Calle']).values.ravel()
dfML['TipoVia_encoded_frequency'] = frequency_encoder_via.fit_transform(dfML['Tipo de vía']).values.ravel()
dfML['Distrito_encoded_frequency'] = frequency_encoder_distrito.fit_transform(dfML['Distrito']).values.ravel()

# Guardar el codificador de frecuencia en un archivo Joblib
# joblib.dump(frequency_encoder, 'frequency_encoder.joblib')

# Inicializar el codificador de objetivo
target_encoder = ce.TargetEncoder(cols=['Calle'])

# Aplicar el target encoding a la columna 'Calle' utilizando la columna 'Lesividad_codificada' como objetivo
dfML['Calle_encoded_target'] = target_encoder.fit_transform(dfML['Calle'], dfML['Lesividad_codificada']).values.ravel()

# Guardar el codificador de objetivo en un archivo Joblib
# joblib.dump(target_encoder, 'target_encoder.joblib')


#### Codificación de impacto
Similar al target encoding, pero utiliza un enfoque bayesiano para estimar las probabilidades de la variable objetivo condicionadas a cada categoría.



##### Tipo de Accidente / Implicado / Estado Meteorológico

In [ ]:
# Inicializar el codificador de objetivo promedio para las variables 'Tipo accidente' e 'Implicado'
impact_encoder_tipo_accidente = ce.TargetEncoder(cols=['Tipo accidente'])
impact_encoder_implicado = ce.TargetEncoder(cols=['Implicado'])
impact_encoder_meteorológico = ce.TargetEncoder(cols=['Estado meteorológico'])

# Aplicar el codificador de objetivo promedio a las variables 'Tipo accidente' e 'Implicado'
dfML['Tipo_accidente_impact_encoded'] = impact_encoder_tipo_accidente.fit_transform(dfML['Tipo accidente'], dfML['Lesividad_codificada'])
dfML['Implicado_impact_encoded'] = impact_encoder_implicado.fit_transform(dfML['Implicado'], dfML['Lesividad_codificada'])
dfML['Estado_meteorológico_impact_encoded'] = impact_encoder_meteorológico.fit_transform(dfML['Estado meteorológico'], dfML['Lesividad_codificada'])



#### One-Hot Encoder
Convierte cada valor único de la columna de edad en una nueva columna binaria (0 o 1), indicando la presencia o ausencia de ese valor en cada fila. Esto puede ser útil si no hay un orden inherente en las categorías y no quieres introducir un supuesto de ordinalidad en tus datos.

In [ ]:
import pandas as pd

# Aplicar one-hot encoding a la columna 'sexo'
dfML = pd.get_dummies(dfML, columns=['Sexo'])

# Obtener las nuevas columnas generadas por get_dummies
nuevas_columnas = dfML.columns[dfML.columns.str.startswith('Sexo_')]

# Convertir True a 1 y False a 0 solo en las nuevas columnas
dfML[nuevas_columnas] = dfML[nuevas_columnas].astype(int)

# Imprimir el DataFrame resultante
dfML

